# 카카오API를 사용하여 호식이두마리치킨 가맹점 찾기
- 가맹점이름(jijum), 전화번호(tel), 주소(addr), 위도(x), 경도(y)
- tinydb를 사용하여 데이터 저장
- folium을 이용하여 지도에 표시
- 구글지도 : 96개 / 네이버지도 : 300개 정도 검색됨, 카카오지도에서 800개이상 지점 확인
- 카카오API로 찾을 때 'total_count'값이 45가 넘으면 46번째부터는 출력 불가능 ★
- rect로 지도를 사각형 범위로 잘라서 지점을 확인하고 'search_count'가 45를 넘으면 해당 지점 4등분 하여 다시 지점 확인

In [1]:
import requests
import folium
import collections
from tinydb import TinyDB, Query

In [2]:
# kakao api를 사용하기 위한 key값과 주소 설정
app_key = 'KakaoAK' + '1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'
params = {
    'query': '호식이두마리치킨',
    'page': 1,
    'rect': '',
}
headers  = {
    'Authorization': app_key
}
resp = requests.get(url, params=params, headers=headers)
# requests 모듈의 json()으로 데이터 확인
# resp.json()

In [3]:
def get_store_list(start_x,start_y,end_x,end_y):
    # 지도상 경계선에 들어가는 값을 줄이기 위한 offset값
    offset = 0.01
    #print(start_x,start_y,end_x,end_y)
    cnt = 1
    # 데이터가 담길 리스트
    resp_list = []
    while True:
        # 사각형 증가값 설정
        jump_x = 0.5
        jump_y = 0.5
        # 카카오 api 사용을 위한 headers와 params 설정
        params = {
            'query': '호식이두마리',
            'page': cnt,
            'rect': f'{start_x-offset},{start_y-offset},{end_x+offset},{end_y+offset}'
        }
        headers  = {
            'Authorization': app_key
        }
        # requests.get으로 값 받아오기
        resp = requests.get(url, params=params, headers=headers)
        
        # 찾은 지점의 갯수 확인
        search_count = resp.json()['meta']['total_count']
        print(search_count)
        
        # 45개 이상이면 해당 지점 4분할 해서 지점 재확인
        if search_count > 45:
            print('big data dividing...')
            dividing_x = (start_x + end_x) / 2
            dividing_y = (start_y + end_y) / 2
            # 1
            resp_list.extend(get_store_list(start_x,start_y,dividing_x,dividing_y))
            # 2 x만 이동
            resp_list.extend(get_store_list(dividing_x,start_y,end_x,dividing_y))
            # 3 y만 이동
            resp_list.extend(get_store_list(start_x,dividing_y,dividing_x,end_y))
            # 4 x,y 전부 이동
            resp_list.extend(get_store_list(dividing_x,dividing_y,end_x,end_y))
            return resp_list
        # 1페이지에 15개의 데이터 출력되므로 1페이지면 resp_list에 저장
        else:
            if resp.json()['meta']['is_end']:
                resp_list.extend(resp.json()['documents'])
                return resp_list
            # 아니면 다음 페이지로 넘어가서 데이터 저장
            else:
                print('paging...')
                cnt += 1
                resp_list.extend(resp.json()['documents'])

# 위도, 경도 찾아오는 함수          
def getLatLng(addr):
    app_key = 'KakaoAK' + ' 1b131358d811397813db74376a179722'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    
    # 불러온 데이터가 정상이면 위도, 경도 리턴
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    
    # 주소에 문제가 있는 경우 맨 뒤를 제거하며 주소로 재검색
    else: 
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)

# 지도의 중간값 얻는 함수
def get_avg_point(x_list, y_list):
    x = sum(x_list) / len(x_list)
    y = sum(y_list) / len(y_list)
    return x, y

# 지도 그리는 함수
def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
    return map_noman

# 지도에 마커찍는 함수
def draw_marker(x, y, tel):
    marker = folium.Marker((x, y), popup=tel)
    icon=folium.Icon(color='blue',icon='glyphicon glyphicon-cutlery')
    icon.add_to(marker)
    return marker

In [4]:
app_key = 'KakaoAK' + '1b131358d811397813db74376a179722'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json'

# 시작 x 좌표 및 증가값
start_x = 126 # ~ 129.7, 3.4
jump_x = 0.5
jump_y = 0.5

# 최종 데이터가 담길 리스트
hosik2_list = []

# 지도를 사각형으로 나누면서 데이터 받아옴
for i in range(1,10):
    end_x = start_x + jump_x
    start_y = 33 # ~ 38.3, 4.9
    for j in range(1,13):
        end_y = start_y + jump_y
        hosik2_list_one = get_store_list(start_x,start_y,end_x,end_y)
        hosik2_list.extend(hosik2_list_one)
        start_y = end_y
    start_x = end_x
hosik2_list

5
1
0
5
1
0
0
1
0
1
0
0
3
6
0
2
32
paging...
32
paging...
32
8
1
5
82
big data dividing...
1
2
26
paging...
26
57
big data dividing...
10
13
17
paging...
17
26
paging...
26
87
big data dividing...
24
paging...
24
64
big data dividing...
25
paging...
25
27
paging...
27
18
paging...
18
3
2
8
0
0
0
0
1
4
2
11
32
paging...
32
paging...
32
34
paging...
34
paging...
34
55
big data dividing...
19
paging...
19
3
33
paging...
33
paging...
33
6
90
big data dividing...
76
big data dividing...
41
paging...
41
paging...
41
16
paging...
16
26
paging...
26
8
5
11
2
4
0
0
0
0
8
5
2
5
4
11
9
2
0
0
0
0
7
17
paging...
17
27
paging...
27
26
paging...
26
7
4
2
1
0
0
0
0
6
50
big data dividing...
18
paging...
18
25
paging...
25
13
6
96
big data dividing...
5
1
82
big data dividing...
45
paging...
45
paging...
45
26
paging...
26
26
paging...
26
12
12
1
6
1
3
2
0
0
0
0
0
54
big data dividing...
43
paging...
43
paging...
43
0
9
5
29
paging...
29
18
paging...
18
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

[{'address_name': '제주특별자치도 제주시 연동 1491-3',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 호식이두마리치킨',
  'distance': '',
  'id': '15220145',
  'phone': '064-773-9922',
  'place_name': '호식이두마리치킨 연동신시가지점',
  'place_url': 'http://place.map.kakao.com/15220145',
  'road_address_name': '제주특별자치도 제주시 국기중길 5-6',
  'x': '126.491685722983',
  'y': '33.4802936485155'},
 {'address_name': '제주특별자치도 제주시 애월읍 애월리 1713-1',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 호식이두마리치킨',
  'distance': '',
  'id': '26902290',
  'phone': '064-799-9909',
  'place_name': '호식이두마리치킨 애월읍내점',
  'place_url': 'http://place.map.kakao.com/26902290',
  'road_address_name': '제주특별자치도 제주시 애월읍 애월로11길 11',
  'x': '126.320324644821',
  'y': '33.465487016673'},
 {'address_name': '제주특별자치도 제주시 노형동 2613-10',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 치킨 > 호식이두마리치킨',
  'distance': '',
  'id': '202

In [5]:
hosik2_list2 = []
# 최종 데이터가 담긴 리스트 중복값 제거
hosik2_list = list(map(dict, collections.OrderedDict.fromkeys(tuple(sorted(d.items())) for d in hosik2_list)))
for i in hosik2_list:
    j = i['place_name']
    tel = i['phone']
    addr = i['address_name']
    x_list = float(i['y'])
    y_list = float(i['x'])
    hosik2_list2.append({'jijum':j, 'tel':tel, 'addr':addr, 'x':x_list, 'y':y_list})
len(hosik2_list2)

822

## 지도에 지점 위치 표시하기

In [6]:
x_list = list(map(lambda i:i['x'], hosik2_list2))
y_list = list(map(lambda i:i['y'], hosik2_list2))
tel_list = list(map(lambda i:i['tel'], hosik2_list2))
hosik2_map = folium.Map(location=get_avg_point(x_list, y_list),zoom_start=7)
for x, y, t in zip(x_list, y_list, tel_list):
    marker = draw_marker(x, y, t)
    marker.add_to(hosik2_map)
hosik2_map

## Tinydb를 사용하여 데이터 저장하기
- 도메인명.tinydb로 저장

In [9]:
# db 연결하기
db = TinyDB('9922(hosik2).tinydb')

# 테이블 생성
table = db.table('chicken')

# 테이블에 데이터 추가하기
for i in hosik2_list2:
    table.insert({'jijum':i['jijum'], 'tel':i['tel'], 'addr':i['addr'], 'x':i['x'], 'y':i['y']})

In [10]:
# 테이블 데이터 확인하기
print(table.all())

[{'jijum': '호식이두마리치킨 연동신시가지점', 'tel': '064-773-9922', 'addr': '제주특별자치도 제주시 연동 1491-3', 'x': 33.4802936485155, 'y': 126.491685722983}, {'jijum': '호식이두마리치킨 애월읍내점', 'tel': '064-799-9909', 'addr': '제주특별자치도 제주시 애월읍 애월리 1713-1', 'x': 33.465487016673, 'y': 126.320324644821}, {'jijum': '호식이두마리치킨 노형뜨란채점', 'tel': '064-744-9929', 'addr': '제주특별자치도 제주시 노형동 2613-10', 'x': 33.4853588784673, 'y': 126.473988762375}, {'jijum': '호식이두마리치킨 외도점', 'tel': '064-711-9904', 'addr': '제주특별자치도 제주시 외도일동 533-14', 'x': 33.4890922966888, 'y': 126.434067707386}, {'jijum': '호식이두마리치킨 제주지역본부', 'tel': '064-712-9920', 'addr': '제주특별자치도 제주시 도평동 641-7', 'x': 33.48743690179725, 'y': 126.45169888127299}, {'jijum': '호식이두마리치킨 용담로터리점', 'tel': '064-713-2177', 'addr': '제주특별자치도 제주시 용담2동 624-12', 'x': 33.5092472688752, 'y': 126.510927987498}, {'jijum': '호식이두마리치킨 용해점', 'tel': '061-276-1818', 'addr': '전남 목포시 용당동 1091-51', 'x': 34.8078720400675, 'y': 126.40023357038}, {'jijum': '호식이두마리치킨 대불점', 'tel': '', 'addr': '전남 영암군 삼호읍 용앙리 1669-1', 'x